In [ ]:
!pip install openai pinecone LangChain

In [ ]:
!pip install --upgrade openai

In [ ]:
import pandas as pd
import openai
import pinecone


In [ ]:
# OpenAI API 키 설정
from pinecone import Pinecone
from openai import OpenAI


openai.api_key = 'sk-proj-p2ti3uV1OWTRF3d5IljCT3BlbkFJGUHpeTHBINuJDMP5x5l3'



pc = Pinecone(api_key="6a58b4b8-48f2-49aa-952f-5e4658e23f69")
index = pc.Index("prod")

In [ ]:
def embedding(data):
    response = openai.embeddings.create(
        input= data,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding


In [ ]:
print(embedding("textract, ocr"))

In [ ]:
def get_summary(desc):
    prompt = f"""
    강의의 내용은 다음과 같습니다.
    "{desc}"
    강의 내용이 위와 같을 때 강의 한줄 소개 글을 작성해주세요
    강의에서 배우는 aws 서비스가 있을 경우 서비스 명과 설명도 간단하게 붙여주세요.
    """
    response =  openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "AWS 및 IT 기술에 특화된 어시스턴트입니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0
    )
    return response.choices[0].message.content


In [ ]:
def get_aws_and_it_keywords_from_desc(desc):
    prompt = f"""
    The user asked the following question:
    "{desc}"

    Provide four AWS service names and four IT technology keywords related to the content.
    Each keyword should be separated by a comma and a space, and the whole output should be in one line.
    Do not include any labels such as "AWS services" or "IT keywords and .
    Exclude "IAM", "AWS", "Amazon" from the AWS services."
    Each Keyword must be an english
    """
    response =  openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant specializing in AWS and IT technology."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        temperature=0
    )
    return response.choices[0].message.content



In [ ]:
df = pd.read_csv('./learning_addcol.csv')


selected_cols = df[['id','title','difficulty','duration', 'desc', 'summary','thumbnail','link','type', 'keywords', ]]



## 그냥 그대로 summary만 임베딩하여 넣었을 경우
vectors = []
for i, row in selected_cols.iterrows():

    vector = {}
    metadata = {}

    vector["metadata"] = metadata
    vector["values"] = ""
    print(i)
    try:
        sum = get_summary(row["desc"])
        keywords = get_aws_and_it_keywords_from_desc(row["desc"])
        selected_cols.at[i, 'summary'] = sum
        keywords = keywords.replace("Amazon","").replace("AWS", "").replace("  ", " ")
        selected_cols.at[i, 'keywords'] = keywords
    except Exception as e:
        selected_cols.to_csv('./updated_learning.csv', index=False)


    #vectors.append(vector)

selected_cols.to_csv('./updated_learning.csv', index=False)








In [ ]:
print(get_aws_and_it_keywords_from_desc("시계열 데이터를 관리해야하는데 관련 강의를 추천해줘"))

In [ ]:
selected_cols.to_csv('./updated_learning.csv', index=False, encoding='utf-8')

In [ ]:
## summary 칼럼 그대로 임베딩
df = pd.read_csv('./learning_test3_utf.csv')


selected_cols = df[['id','title','difficulty','duration', 'desc', 'summary','thumbnail','link','type', 'keywords', ]]



## 그냥 그대로 summary만 임베딩하여 넣었을 경우
vectors = []
for i, row in selected_cols.iterrows():

    vector = {}
    metadata = {}

    metadata["id"] = str(row["id"])
    vector["values"] = embedding(row['keywords'].replace("AWS", "").replace("Amazon",""))
    metadata["keywords"] = [s.strip() for s in row["keywords"].split(",")]
    vector["metadata"] = metadata
    vector["id"] = str(row["id"])+"_learning"






    vectors.append(vector)




print(vectors[0])







In [ ]:
## sumarry와 키워드 칼럼 합쳐서 임베딩

In [ ]:
print(vectors[0])

In [ ]:
index.upsert(
    vectors=vectors,
    namespace="learning"
)

In [ ]:
index.query(
    vector = embedding("ec2 초급강의"),
    top_k=3
    namespace="learning"

)

In [ ]:
print(embedding("ECS, EKS, Fargate, App Runner, Kubernetes, Docker, Containerization, Orchestration, 컨테이너 오케스트레이션 중급 과정"))

In [ ]:
print(get_aws_and_it_keywords_from_desc("컨테이너 오케스트레이션 초급 과정"))